# Python adaptation of Mike's cpu_vah code

Planning phase:
1. Need to have input parameters: since this will start off as a Jupyter notebook, I will not store them in a separete file.
1. Translate hydrodynamic evolution code to python and cut out all the unnecessary Trento and freezeout surface stuff
1. Include an option to include both analytic solution and numberical solution for background evolution
1. Generate plots 

For now I will cut out all the code dealing with nonconformal systems

In [1]:
import sys

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

%matplotlib inline
plt.style.use('classic')

In [3]:
import matplotlib.font_manager
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
rc('text', usetex=True)

In [4]:
from hydro_simulation import Hydro_Sim, hydro_parameters, lattice_parameters
from hydro_model import Hydro_Model

In [5]:
# Hydro model to use
hydro_model = 1        # 1 - Mueller-Israel-Stewart Theory
                       # 2 - MIS with linear fluctations

In [6]:
# input parameters for hydro calculations
# =============================
''' Viscosity parameters '''
g_etas_aL       = -0.776   # a_low [GeV^-1],         default: -0.778
g_etas_aH       = 0.37     # a_high [GeV^-1]         default: 0.37
g_etas_Tk_GeV   = 0.223    # T_eta [GeV]             default: 0.223
g_etas_etask    = 0.096    # (eta/s)_kink            default: 0.096
g_etas_min      = 0.01     # lower cutoff (eta/s)(T) default: 0.01
g_constant_etas = 0.2      # value for eta/s = const defualt: 0.2
g_eta_bar       = 5 / (4 * np.pi)

''' Hydrodynamic parameters '''
g_tau_initial               = 0.125   # start time [fm/c]             default: 0.05
g_freezeout_temperature_GeV = 0.136   # stop temp [GeV]               default: 0.136
g_temperature_etas          = 1       # shear viscosity model
                                      # 0: const, 1: temperature dep  default: 1
g_energy_initial            = 15.     # initial energy density [GeV]  default: 15
g_shear_initial             = 0.314   # initial shear stress [GeV]    defautl: 0.314
g_energy_min                = 1.0e-1  # energy density cutoff [fm^-4] default: 1.e-1
g_flux_limiter              = 1.8     # flux limiter in KT algo       default: 1.8
g_regulation_scheme         = 1       # switch for regulation scheme  default: 1
g_dynamical_variables       = 14      # components of Tmn that evolve default: 14
                                      # number of components for hydro_variables class

''' Spatial grid and time step '''
g_lattice_points_x    = 5       # custum grid point Nx           default: 281
g_lattice_points_y    = 5       # costum grid point Ny           default: 281
g_lattice_points_eta  = 5       # costum grid point Neta         default: 1
g_lattice_spacing_x   = 0.1     # costum lattice spacing dx [fm] default: 0.1
g_lattice_spacing_y   = 0.1     # costum lattice spacing dy [fm] default: 0.1
g_lattice_spacing_eta = 0.1     # costum lattice space deta [1]  default: 0.1
g_adaptive_time_step  = 1       # switch for adaptive time step  
                                # 1: on, 0: off                  default: 1
g_delta_0             = 0.004   # control for adaptive time step default: 0.004 
g_alpha               = 0.5     # control for adaptive time step default: 0.5
g_fixed_time_step     = 0.0125  # value for dt = const [fm/c]    default: 0.0125
g_max_time_steps      = 2000    # code fails if exceeded         default: 2000
g_alpha               = 0.5     # safety parameter for time step default: 0.5 \in [0,1]
g_delta_0             = 0.004   # error tolerance                default: 0.004

In [7]:
hydro = hydro_parameters(
    g_tau_initial,
    g_energy_initial,
    g_shear_initial,
    g_freezeout_temperature_GeV,
    g_energy_min,
    g_flux_limiter,
    g_regulation_scheme,
    g_dynamical_variables,
    g_temperature_etas,
    g_eta_bar,
    g_constant_etas,
    g_etas_min,
    g_etas_aL,
    g_etas_aH,
    g_etas_etask,
    g_etas_Tk_GeV
)

lattice = lattice_parameters(
    g_lattice_points_x,
    g_lattice_points_y,
    g_lattice_points_eta,
    g_lattice_spacing_x,
    g_lattice_spacing_y,
    g_lattice_spacing_eta,
    g_adaptive_time_step,
    g_delta_0,
    g_alpha,
    g_fixed_time_step,
    g_max_time_steps,
)

In [8]:
model = Hydro_Model(model_flag=hydro_model, eta_s=hydro.eta_bar)
sim = Hydro_Sim(hydro=hydro, lattice=lattice)

In [ ]:
%%time
sim.run_hydro(model)

Begin simulation.
Allocating memory.
Setting inital conditions.
Time step: 0
Time step: 1


C:\Users\gil-c\Documents\Heinz_Research\TeX-Docs\Rough-Drafts\Anisotropic-Dispersion-Relations\Code\VH_Aniso_IC\hydro_simulation.py:1159: RuntimeWarning: divide by zero encountered in double_scalars
  dt_CFL = np.fmin(dt_CFL, np.fmin(dx / ax, np.fmin(dy / ay, deta / an)))


Time step: 2
Time step: 3
Time step: 4
Time step: 5
Time step: 6
Time step: 7
Time step: 8
Time step: 9
Time step: 10
Time step: 11
Time step: 12
Time step: 13
Time step: 14
Time step: 15
Time step: 16
Time step: 17
Time step: 18
Time step: 19
Time step: 20
Time step: 21
Time step: 22
Time step: 23
Time step: 24
Time step: 25
Time step: 26
Time step: 27
Time step: 28
Time step: 29
Time step: 30
Time step: 31
Time step: 32
Time step: 33
Time step: 34
Time step: 35
Time step: 36
Time step: 37
Time step: 38
Time step: 39
Time step: 40
Time step: 41
Time step: 42
Time step: 43
Time step: 44
Time step: 45
Time step: 46
Time step: 47
Time step: 48
Time step: 49
Time step: 50
Time step: 51
Time step: 52
Time step: 53
Time step: 54
Time step: 55
Time step: 56
Time step: 57
Time step: 58
Time step: 59
Time step: 60
Time step: 61
Time step: 62
